In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from openpyxl import Workbook
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# Optional: Install selenium-stealth if needed (uncomment to use)
# pip install selenium-stealth
try:
    from selenium_stealth import stealth
except ImportError:
    stealth = None
    print("Warning: selenium-stealth not installed. Some anti-bot measures may still block the script.")

In [3]:
# Read the input Excel file
df = pd.read_excel("D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/cvgeniusITlink.xlsx")

# Initialize list to store all results
all_results = []

In [4]:
# Function to clean text: remove newlines, extra spaces, and special characters
def clean_text(text):
    # Remove newlines, tabs, and special characters
    text = re.sub(r'[\n\r\t]+', ' ', text)
    # Remove non-breaking spaces and other special characters
    text = re.sub(r'\xa0', ' ', text)
    # Collapse multiple spaces into a single space
    text = re.sub(r'\s+', ' ', text)
    # Strip leading/trailing spaces
    return text.strip()

In [5]:
# Function to set up WebDriver with anti-bot measures
def setup_driver(max_retries=2):
    for attempt in range(max_retries + 1):
        try:
            chrome_options = Options()
            # Use real browser (non-headless mode) with visibility
            chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
            chrome_options.add_argument("--disable-blink-features=AutomationControlled")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("--disable-extensions")
            chrome_options.add_argument("--disable-infobars")
            
            # Set up the driver with the latest ChromeDriver version
            driver = webdriver.Chrome(
                service=Service(ChromeDriverManager().install()),
                options=chrome_options
            )
            
            # Apply stealth settings if available
            if stealth:
                stealth(driver,
                        languages=["en-US", "en"],
                        vendor="Google Inc.",
                        platform="Win32",
                        webgl_vendor="Intel Inc.",
                        renderer="Intel Iris OpenGL Engine",
                        fix_hairline=True,
                )
            
            print(f"Info: WebDriver initialized successfully on attempt {attempt + 1}")
            return driver
        
        except WebDriverException as e:
            print(f"Error initializing WebDriver on attempt {attempt + 1}: {str(e)}")
            if attempt == max_retries:
                raise Exception(f"Failed to initialize WebDriver after {max_retries + 1} attempts: {str(e)}")
            time.sleep(2)  # Wait before retrying

# Function to handle pop-ups (e.g., cookie banners)
def handle_popups(driver, url):
    try:
        # Look for common cookie banner buttons
        accept_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept') or contains(text(), 'Agree') or contains(text(), 'OK')]"))
        )
        accept_button.click()
        print(f"Info: Closed cookie banner for {url}")
        time.sleep(1)  # Wait for the banner to close
    except (NoSuchElementException, TimeoutException):
        print(f"Info: No cookie banner found for {url}")
    except Exception as e:
        print(f"Warning: Error closing pop-up for {url}: {str(e)}")

# Function to crawl resume content from the specified structure
def crawl_resume_content(url, category, max_retries=1):
    for attempt in range(max_retries + 1):
        driver = None
        try:
            # Set up WebDriver
            driver = setup_driver(max_retries=2)
            
            # Navigate to the URL
            driver.get(url)
            print(f"Info: Successfully loaded {url} on attempt {attempt + 1}")
            
            # Wait for the main wrapper to load (timeout set to 20 seconds)
            try:
                accordion_item = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "accordion-item.accordion-item-static"))
                )
                print(f"Info: Accordion item detected for {url}")
            except TimeoutException:
                # Check for CAPTCHA or anti-bot page
                try:
                    captcha = driver.find_element(By.XPATH, "//*[contains(text(), 'CAPTCHA') or contains(text(), 'verify') or contains(text(), 'robot')]")
                    print(f"Warning: CAPTCHA detected for {url}. Please solve the CAPTCHA manually in the browser window.")
                    time.sleep(30)  # Pause to allow manual CAPTCHA solving
                    # Re-check for content after CAPTCHA solving
                    accordion_item = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "accordion-item.accordion-item-static"))
                    )
                    print(f"Info: Accordion item detected after CAPTCHA solving for {url}")
                except (NoSuchElementException, TimeoutException):
                    print(f"Error: Timeout waiting for accordion item for {url} after {attempt + 1} attempts.")
                    if attempt == max_retries:
                        driver.quit()
                        return []
                    continue
            
            # Handle pop-ups (e.g., cookie banners)
            handle_popups(driver, url)
            
            # Navigate to <div class="entry-content">
            try:
                entry_content = accordion_item.find_element(By.CLASS_NAME, "entry-content")
            except NoSuchElementException:
                print(f"Error: Could not find 'entry-content' for {url}.")
                driver.quit()
                return []
            
            # Navigate to <div class="copy-div">
            try:
                copy_div = entry_content.find_element(By.CLASS_NAME, "copy-div")
            except NoSuchElementException:
                print(f"Error: Could not find 'copy-div' for {url}.")
                driver.quit()
                return []
            
            # Initialize list to store resume content
            resume_content = []
            
            # Find all direct children that are <p> or <ul> tags
            elements = copy_div.find_elements(By.XPATH, "./*[self::p or self::ul]")
            
            for element in elements:
                tag_name = element.tag_name
                
                # Extract <p> (includes content from child tags like <strong>, <em>)
                if tag_name == "p":
                    text = clean_text(element.text)
                    if text:
                        resume_content.append(text)
                
                # Extract from <ul> (includes <li> and their child content)
                elif tag_name == "ul":
                    try:
                        li_items = element.find_elements(By.TAG_NAME, "li")
                        for li in li_items:
                            text = clean_text(li.text)
                            if text:
                                resume_content.append(text)
                    except NoSuchElementException:
                        pass
            
            # Join all content with a single space
            full_content = " ".join(resume_content)
            
            # Since CVGenius seems to have only one CV per page, return a single entry
            driver.quit()
            if full_content:
                return [{"Category": category, "Resume": full_content}]
            return []
        
        except WebDriverException as e:
            print(f"Error on attempt {attempt + 1} for {url}: {str(e)}")
            if attempt == max_retries:
                print(f"Error: Failed to crawl {url} after {max_retries + 1} attempts.")
                if driver:
                    driver.quit()
                return []
        except Exception as e:
            print(f"Unexpected error on attempt {attempt + 1} for {url}: {str(e)}")
            if attempt == max_retries:
                print(f"Error: Failed to crawl {url} after {max_retries + 1} attempts.")
                if driver:
                    driver.quit()
                return []
        finally:
            if driver:
                try:
                    driver.quit()
                except:
                    pass

# Function to process a single row (for parallel execution)
def process_row(row):
    category = row["Category"]
    url = row["Resume_link"]
    print(f"Crawling resume for {category}...")
    resume_contents = crawl_resume_content(url, category, max_retries=1)
    return resume_contents

In [6]:
# Use ThreadPoolExecutor for parallel crawling (2 workers for stability)
with ThreadPoolExecutor(max_workers=2) as executor:
    # Submit all tasks
    future_to_row = {executor.submit(process_row, row): row for _, row in df.iterrows()}
    
    # Collect results as they complete
    for future in as_completed(future_to_row):
        results = future.result()
        all_results.extend(results)
        # Small delay to avoid overwhelming the server
        time.sleep(0.5)

Crawling resume for Computer science...
Crawling resume for Cyber security...
Info: WebDriver initialized successfully on attempt 1
Info: WebDriver initialized successfully on attempt 1
Info: Successfully loaded https://cvgenius.com/cv-examples/cyber-security-cv on attempt 1
Info: Successfully loaded https://cvgenius.com/cv-examples/computer-science-cv on attempt 1
Info: Accordion item detected for https://cvgenius.com/cv-examples/cyber-security-cv
Info: Accordion item detected for https://cvgenius.com/cv-examples/computer-science-cv
Info: No cookie banner found for https://cvgenius.com/cv-examples/cyber-security-cv
Info: No cookie banner found for https://cvgenius.com/cv-examples/computer-science-cv
Crawling resume for Data analyst...
Crawling resume for Data engineer ...
Info: WebDriver initialized successfully on attempt 1
Info: Successfully loaded https://cvgenius.com/cv-examples/data-analyst-cv on attempt 1
Info: Accordion item detected for https://cvgenius.com/cv-examples/data-an

In [7]:
# Create a DataFrame with the results
result_df = pd.DataFrame(all_results)

# Ensure the DataFrame has the correct columns in the right order
result_df = result_df[["Category", "Resume"]]

In [8]:
result_df.head(5)

,Category,Resume
0,Cyber security,PERSONAL STATEMENT Diligent and technically as...
1,Computer science,PROFESSIONAL SUMMARY Analytical and solution-o...
2,Data analyst,PERSONAL STATEMENT Innovative and process-orie...
3,Data engineer,PERSONAL STATEMENT Data engineer with 4 years ...
4,Data science,PERSONAL STATEMENT Google and IBM certified Da...


In [9]:
# Save to Excel
result_df.to_excel("D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/NewITData/finalcvgeniusITlink.xlsx", index=False)
print("Crawling complete. Results saved to Crawled_Resumes.xlsx")

Crawling complete. Results saved to Crawled_Resumes.xlsx
